# What do I want?

Previously in `HSC_COSMOS_filtering.ipynb` I tested out some basic classifiers to get a smaller sample set, while still keeping completeness high.  I tested two basic classifiers: a RandomForest classifier and a Logistic Regression classifier.

For my training data, I started by getting objects and labels from COSMOS. For input features, I then matched those COSMOS galaxies to their nearest HSC counterpart. I then used HSC i-band magnitude, along with HSC g-r, r-i, i-z, z-y colors.

Choosing some arbitrary thresholds, I got similar results for the Random Forest and the Logistic Regression classifiers. In this notebook I'll look at the full ROC curves for both classifiers, in hopes of better understanding my results.

# Code

In [ ]:
# give access to importing dwarfz
import os, sys
dwarfz_package_dir = os.getcwd().split("dwarfz")[0]
if dwarfz_package_dir not in sys.path:
    sys.path.insert(0, dwarfz_package_dir)

import dwarfz
    
# back to regular import statements

import numpy as np

In [ ]:
COSMOS_filename = os.path.join(dwarfz.data_dir_default, "COSMOS_reference.sqlite")
COSMOS = dwarfz.datasets.COSMOS(COSMOS_filename)

In [ ]:
HSC_filename = os.path.join(dwarfz.data_dir_default, "HSC_COSMOS_median_forced.sqlite3")
HSC = dwarfz.datasets.HSC(HSC_filename)

In [ ]:
COSMOS.df.shape

In [ ]:
HSC.df.shape

In [ ]:
matches_filename = os.path.join(dwarfz.data_dir_default, "matches.sqlite3")
matches_df = dwarfz.matching.Matches.load_from_filename(matches_filename)

In [ ]:
combined = matches_df[matches_df.match].copy()
combined["ra"]       = COSMOS.df.loc[combined.index].ra
combined["dec"]      = COSMOS.df.loc[combined.index].dec
combined["photo_z"]  = COSMOS.df.loc[combined.index].photo_z
combined["log_mass"] = COSMOS.df.loc[combined.index].mass_med

photometry_cols = [
    "gcmodel_flux","gcmodel_flux_err","gcmodel_flux_flags",
    "rcmodel_flux","rcmodel_flux_err","rcmodel_flux_flags",
    "icmodel_flux","icmodel_flux_err","icmodel_flux_flags",
    "zcmodel_flux","zcmodel_flux_err","zcmodel_flux_flags",
    "ycmodel_flux","ycmodel_flux_err","ycmodel_flux_flags",
]

for col in photometry_cols:
    combined[col] = HSC.df.loc[combined.catalog_2_ids][col].values

In [ ]:
low_z    = (combined.photo_z  < .15)
low_mass = (combined.log_mass < 9)

# Create classification labels

Class A: matched **and** (low redshift + low mass)

Class B: matched **but not** (low redshift + low mass)

In [ ]:
class_a =  (low_z & low_mass)
class_b = ~(low_z & low_mass)

In [ ]:
combined["low_z_low_mass"] = class_a
combined.head()

## Turn fluxes into rough colors
Yes, I know these aren't exactly the right colors since I'm not including zero-points, but that shouldn't affect the results.

(When I get a chance, I'll re-download the dataset so that it includes magnitudes not just fluxes)

In [ ]:
combined["g_minus_r"] = -.4*np.log10(combined["gcmodel_flux"] / combined["rcmodel_flux"])
combined["r_minus_i"] = -.4*np.log10(combined["rcmodel_flux"] / combined["icmodel_flux"])
combined["i_minus_z"] = -.4*np.log10(combined["icmodel_flux"] / combined["zcmodel_flux"])
combined["z_minus_y"] = -.4*np.log10(combined["zcmodel_flux"] / combined["ycmodel_flux"])

For now, filter out bad photometry. Later I could consider passing this into the classifier, as an imputed/sentinel value

In [ ]:
mask =    np.isfinite(combined["g_minus_r"]) & np.isfinite(combined["r_minus_i"]) \
        & np.isfinite(combined["i_minus_z"]) & np.isfinite(combined["z_minus_y"]) \
        & np.isfinite(combined["icmodel_flux"]) \
        & (~combined.gcmodel_flux_flags) & (~combined.rcmodel_flux_flags) \
        & (~combined.icmodel_flux_flags) & (~combined.zcmodel_flux_flags) \
        & (~combined.ycmodel_flux_flags)

combined = combined[mask]

combined["log_icmodel_flux"] = np.log10(combined["icmodel_flux"])

In [ ]:
combined.shape

In [ ]:
features = combined.loc[:,["g_minus_r", "r_minus_i", "i_minus_z", "z_minus_y",
                              "log_icmodel_flux"]]

target = combined.loc[:,["low_z_low_mass"]]

In [ ]:
target.mean()

# Build Classifiers

## Partition training and testing sets

In [ ]:
testing_fraction = .1
test_set_indices = np.random.choice(target.index.values, 
                                    replace=False,
                                    size=int(testing_fraction*target.size)
                                   )

training_set_indices = np.array(list(set(target.index.values) - set(test_set_indices)))

features_train = features.loc[training_set_indices]
features_test  = features.loc[test_set_indices]

target_train   = target.loc[training_set_indices]
target_test    = target.loc[test_set_indices]

true_a =  target_test.values.flatten()
true_b = ~target_test.values.flatten()

In [ ]:
def get_classification_characteristics(target_prob, threshold_prob, verbose=False):

    target_prediction = (target_prob > threshold_prob)
    
    prediction_a =  target_prediction
    prediction_b = ~target_prediction
    
    completeness = (true_a & prediction_a).sum() / (true_a).sum() 
    
    purity = (true_a & prediction_a).sum() / (prediction_a).sum() 
    
    sample_size_reduction = prediction_a.size / prediction_a.sum()
    
    true_positives  = np.sum(true_a & prediction_a)
    false_positives = np.sum(true_b & prediction_a)
    
    true_negatives  = np.sum(true_b & prediction_b)
    false_negatives = np.sum(true_a & prediction_b)
    
    true_positive_rate = true_positives / true_a.sum()
    false_positive_rate = false_positives / true_b.sum()
    
    if verbose:
        print("completeness:          ", completeness)
        print("purity:                ", purity)
        print("sample_size_reduction: ", sample_size_reduction)
        print("true  positive rate:   ", true_positive_rate)
        print("false positive rate:   ", false_positive_rate)
        
    return {
        "completeness": completeness,
        "purity": purity,
        "sample_size_reduction": sample_size_reduction,
        "threshold_prob": threshold_prob,
        "true_positive_rate": true_positive_rate,
        "false_positive_rate": false_positive_rate,
           }

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier_RF = RandomForestClassifier()
classifier_RF = classifier_RF.fit(features_train, target_train.values.flatten())

target_prob_RF = classifier_RF.predict_proba(features_test)[:,1]
print("min prob: ", target_prob_RF.min())
print("max prob: ", target_prob_RF.max())


In [ ]:
get_classification_characteristics(target_prob_RF, .01, verbose=True)

In [ ]:
threshold_probs = np.linspace(0, 1, num=100)[1:-1]
results_RF = [get_classification_characteristics(target_prob_RF, threshold_prob)
              for threshold_prob in threshold_probs]

In [ ]:
completenesses_RF         = [result["completeness"] for result in results_RF]
purities_RF               = [result["purity"] for result in results_RF]
sample_size_reductions_RF = [result["sample_size_reduction"] for result in results_RF]
true_positive_rates_RF    = [result["true_positive_rate"] for result in results_RF]
false_positive_rates_RF   = [result["false_positive_rate"] for result in results_RF]

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier_LR = LogisticRegression(class_weight="balanced")
classifier_LR = classifier_LR.fit(features_train, np.array(target_train.values.flatten(), dtype=int))

target_prob_LR = classifier_LR.predict_proba(features_test)[:,1]
print("min prob: ", target_prob_LR.min())
print("max prob: ", target_prob_LR.max())

In [ ]:
get_classification_characteristics(target_prob_LR, .01, verbose=True)

In [ ]:
threshold_probs = np.linspace(0, 1)[1:-1]
results_LR = [get_classification_characteristics(target_prob_LR, threshold_prob)
              for threshold_prob in threshold_probs]

In [ ]:
completenesses_LR         = [result["completeness"] for result in results_LR]
purities_LR               = [result["purity"] for result in results_LR]
sample_size_reductions_LR = [result["sample_size_reduction"] for result in results_LR]
true_positive_rates_LR    = [result["true_positive_rate"] for result in results_LR]
false_positive_rates_LR   = [result["false_positive_rate"] for result in results_LR]


# Get specific galaxies

In [ ]:
best_dwarfs_args  = np.argpartition(target_prob_RF, target_prob_RF.size-100)[-100:]
worst_dwarfs_args = np.argpartition(target_prob_RF, 100)[:100]

best_dwarfs_ids_cosmos  = target_test.iloc[best_dwarfs_args].index
worst_dwarfs_ids_cosmos = target_test.iloc[worst_dwarfs_args].index

best_dwarf_ids_hsc = combined.loc[best_dwarfs_ids_cosmos].catalog_2_ids
worst_dwarf_ids_hsc = combined.loc[worst_dwarfs_ids_cosmos].catalog_2_ids

In [ ]:
random_ids_cosmos = np.random.choice(training_set_indices,
                              replace=False,
                              size=100,
                             )

random_ids_hsc = combined.loc[random_ids_cosmos].catalog_2_ids

## Check: do any HSC ids overlap?
By design the COSMOS ids shouldn't overlap, but the COSMOS id -> HSC id mapping isn't necessarily unique.

In [ ]:
set(best_dwarf_ids_hsc.values) & set(worst_dwarf_ids_hsc.values)

In [ ]:
set(best_dwarf_ids_hsc.values) & set(random_ids_hsc.values)

In [ ]:
set(worst_dwarf_ids_hsc.values) & set(random_ids_hsc.values)

### Do they give reasonable dwarf fractions?

In [ ]:
combined.loc[best_dwarfs_ids_cosmos].low_z_low_mass.mean()

In [ ]:
combined.loc[worst_dwarfs_ids_cosmos].low_z_low_mass.mean()

In [ ]:
combined.loc[random_ids_cosmos].low_z_low_mass.mean()

### Save the indices to disk
(but only if they don't already exist)

In [ ]:
data_dir_quick_sample = os.path.join(dwarfz.data_dir_default, "quick_sample")
if not os.path.exists(data_dir_quick_sample):
    os.mkdir(data_dir_quick_sample)

    np.savetxt(os.path.join(data_dir_quick_sample,"ids_best.csv"),   best_dwarf_ids_hsc.values,  fmt="%d")
    np.savetxt(os.path.join(data_dir_quick_sample,"ids_worst.csv"),  worst_dwarf_ids_hsc.values, fmt="%d")
    np.savetxt(os.path.join(data_dir_quick_sample,"ids_random.csv"), random_ids_hsc.values,      fmt="%d")

# What image size do I need?

For this, you'll need to use `data.get_shapes.ipynb` to query + store the object shapes from the remote database.

# Build a list to send to Song

In [ ]:
ids_best   = np.loadtxt(os.path.join(data_dir_quick_sample,"ids_best.csv"),
                        dtype=int)
ids_worst  = np.loadtxt(os.path.join(data_dir_quick_sample,"ids_worst.csv"),
                        dtype=int)
ids_random = np.loadtxt(os.path.join(data_dir_quick_sample,"ids_random.csv"),
                        dtype=int)

## Check if any HSC ids are duplicated

In [ ]:
assert( len(set(best_dwarf_ids_hsc)) == 100 )

In [ ]:
assert( len(set(worst_dwarf_ids_hsc)) == 100 )

In [ ]:
assert( len(set(random_ids_hsc)) == 100 )

### But are multiple cosmos galaxies matched to any of those HSC id's?
This will be a problem because there will be two masses / redshifts attached to a given HSC example.

In [ ]:
df_best = combined[combined.catalog_2_ids.isin(best_dwarf_ids_hsc)]
df_best.shape

In [ ]:
df_worst = combined[combined.catalog_2_ids.isin(worst_dwarf_ids_hsc)]
df_worst.shape

In [ ]:
df_random = combined[combined.catalog_2_ids.isin(random_ids_hsc)]
df_random.shape

# Disambiguate matches
Simply select the closest of the COSMOS galaxies, and discard all others

In [ ]:
from collections import Counter

def copy_and_filter_duplicates(df_old, verbose=False):
    df = df_old.copy()
    counts = Counter(df.catalog_2_ids)
    
    ambiguous_galaxy_hsc_ids = []
    for hsc_id in counts:
        if counts[hsc_id] > 1:
            ambiguous_galaxy_hsc_ids.append(hsc_id)
            if verbose:
                print(hsc_id, counts[hsc_id])
            
            ambiguous_matches = df[df.catalog_2_ids == hsc_id]
            better_match_cosmos_id = ambiguous_matches.sep.argmin()
            worse_match_ids = set(ambiguous_matches.index) - set([better_match_cosmos_id])
            
            for worse_match_id in worse_match_ids:
                df = df[df.index != worse_match_id ]
        
    return df   

In [ ]:
df_best = copy_and_filter_duplicates(df_best, verbose=True)

In [ ]:
df_worst = copy_and_filter_duplicates(df_worst, verbose=True)

In [ ]:
df_random = copy_and_filter_duplicates(df_random, verbose=True)

## Create a csv file for Song

In [ ]:
with open(os.path.join(data_dir_quick_sample,"galaxies_for_song.csv"),
          mode="w") as f:
    f.write("# object_id,ra,dec,z\n")
    
    kwargs = dict(
        header=None,
        index=False,
        float_format="%lf",
    )
    
    columns = ["catalog_2_ids", "ra", "dec", "photo_z"]
        
    df_best[columns].to_csv(f, **kwargs)
    
    df_worst[columns].to_csv(f, **kwargs)
    
    df_worst[columns].to_csv(f, **kwargs)


In [ ]:
!head {data_dir_quick_sample}/galaxies_for_song.csv